In [1]:
import pandas as pd
import numpy as np
import random

## Download file 

In [2]:
## Test per capire se il file .csv è stato caricato correttamente
## NB: fino a 1 minuto di run
csv_original_df = 'Data/Reviews_With_Metadata.csv' 
Reviews_With_Metadata = pd.read_csv(csv_original_df)
Reviews_With_Metadata

/var/folders/60/_hldy1910cj16rlhrx9tjd_40000gn/T/ipykernel_14032/1266386512.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  Reviews_With_Metadata = pd.read_csv(csv_original_df)


,product_asin,reviewer_id,reviewer_name,review_score,review_time,review_time_unix,review_summary,review_upvote,review_text,product_title,product_brand
0,B004UDB9SA,A2U4YOLR0BMKHL,johan montaez,5,2015-04-29 00:00:00,1430265600,Five Stars,NaN,excellent,Resident Evil: Operation Raccoon City - Playst...,by\n \n Capcom
1,B004UDB9SA,A35698SZOE5PTZ,The Big LEO,4,2015-04-07 00:00:00,1428364800,Four Stars,NaN,GOOD GAME,Resident Evil: Operation Raccoon City - Playst...,by\n \n Capcom
2,B004UDB9SA,ATFTEK383YJ28,tim l.,5,2015-04-03 00:00:00,1428019200,Five Stars,NaN,Ok,Resident Evil: Operation Raccoon City - Playst...,by\n \n Capcom
3,B004UDB9SA,A3JE7R2WFCEBQN,Randy BoBandy,1,2015-03-26 00:00:00,1427328000,Yet another terrible spin-off cashing in on th...,NaN,Capcom is one of the greediest and lazy game d...,Resident Evil: Operation Raccoon City - Playst...,by\n \n Capcom
4,B004UDB9SA,A3MH8YUUJCJHIS,Glen49,5,2015-03-22 00:00:00,1426982400,Five Stars,NaN,OK,Resident Evil: Operation Raccoon City - Playst...,by\n \n Capcom
...,...,...,...,...,...,...,...,...,...,...,...
2467152,B012B6XWA0,A2HRN7YQ5TQ5VT,Andrew,2,2015-10-22 00:00:00,1445472000,To small,NaN,This mouse is too small. I would stay away fr...,BenQ Zowie ZA12 E-Sports Ambidextrous Optical ...,BenQ
2467153,B012B6XWA0,A2CQVQN24YXURG,Lacivious,4,2015-10-17 00:00:00,1445040000,"Grab your Snow Shovel, and your pajamas, and s...",3,So i just got these today. Wow i already have ...,BenQ Zowie ZA12 E-Sports Ambidextrous Optical ...,BenQ
2467154,B012B6XWA0,A1TCXZC1ZCJELI,Vu,5,2015-10-16 00:00:00,1444953600,I have nothing but good things to say about th...,2,Regarding ZA13:\nI have nothing but good thing...,BenQ Zowie ZA12 E-Sports Ambidextrous Optical ...,BenQ
2467155,B012B6XWA0,A1OZJI75NABCPM,Sven,3,2015-09-20 00:00:00,1442707200,I was completely happy with this mouse for a f...,4,I was completely happy with this mouse for a f...,BenQ Zowie ZA12 E-Sports Ambidextrous Optical ...,BenQ


# Campi che descrivono la recensione:
- product_asin: codice ASIN univoco del prodotto (asin)
- reviewer_id: identificatore univoco della persona che ha scritto la recensione (reviewerID)
- reviewer_score: voto assegnato nell’ambito della recensione, nell’intervallo [1, 5] (overall)
- review_time: data della recensione
- review_time_unix: data della recensione come millisecondi trascorsi dal 1970
- review_summary: riassunto del testo della recensione (summary)
- review_upvote: numero di volte in cui la recensione è stata indicata come utile da altre persone (vote)
- review_text: il testo della recensione (reviewText)
- product_title: nome del prodotto recensito (title) j. product_brand: brand del prodotto recensito (brand)
- product_brand: brand del prodotto recensito (brand)

## Searching for 15 review:
- Nome e marca del prodotto non devono avere un valore nullo
- Le recensioni devono essere state indicate come utili da altre persone almeno 1 volta
- Ciascuna recensione deve far riferimento ad un prodotto diverso
- Generate per ogni riga del DataFrame ricevuto un identificatore univoco per la coppia recensione / prodotto

In [8]:
## Genera un nuovo file .csv con solo recensioni utili per lo HIT
## Ogni run crea un file differente
def seclectReview():

    #filtra il databese: elimina righe con valori NaN nelle date colonne 
    #                    elimina recensioni prodotti recensiti più volte
    Reviews = Reviews_With_Metadata.dropna(subset=['product_title', 'product_brand', 'review_upvote'])  
    Reviews = Reviews.drop_duplicates(subset=['product_asin'])
    
    #da un df con solo entry idonee ne selezione un sottoinsieme
    df_HIT_reviews = Reviews.sample(n=15)   
    product_brand = df_HIT_reviews['product_brand'].str.replace("by\n    \n    ", "")
    review_text = df_HIT_reviews['review_text'].str.replace("\n", " ")
    df_HIT_reviews['product_brand'] = product_brand
    df_HIT_reviews['review_text'] = review_text
    df_HIT_reviews.to_csv(r'df_HIT_reviews.csv', index=False)

In [14]:
seclectReview()